In [1]:
import os
import sys

sys.path.append("..")

from itertools import product
import pandas as pd
import kornia.augmentation as K
import numpy as np
import torch
import torch.nn as nn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

from src.models import get_model_by_name
from src.datasets.ucm import UCMMinimal
from src.utils import extract_features
from tqdm import tqdm

device = torch.device("cuda:1")

In [2]:
# Fit and save features
model_names = [
    "resnet50_pretrained_seco",
    "resnet50_pretrained_moco",
    "resnet18_pretrained_moco",
    "resnet50_pretrained_imagenet",
    "resnet50_randominit",
    "imagestats",
    "mosaiks_512_3",
]
rgbs = [True]
sizes = [64, 256]
normalization_methods = ["divide", "standardization"]
use_both_train_val = [True, False]

columns = {
    "model_name": [],
    "rgb": [],
    "size": [],
    "normalization": [],
    "use_train_val": [],
    "accuracy": [],
    "accuracy_scaler": [],
}

for model_name, rgb, size, normalization, use_both in tqdm(product(model_names, rgbs, sizes, normalization_methods, use_both_train_val), total=7*2*2*2):
    transforms = nn.Sequential(K.Resize(size)).to(device)
    dm = UCMMinimal(
        root="../data/ucm/",
        normalization_method=normalization,
        batch_size=64,
        num_workers=8,
        use_both_trainval=use_both,
    )
    dm.setup()


    model = get_model_by_name(model_name, rgb, device=device)

    x_train, y_train = extract_features(
        model, dm.train_dataloader(), device, transforms=transforms, verbose=False
    )
    x_test, y_test = extract_features(
        model, dm.test_dataloader(), device, transforms=transforms, verbose=False
    )

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(x_train, y_train)
    acc = knn_model.score(x_test, y_test)

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    knn_model = KNeighborsClassifier(n_neighbors=5)
    knn_model.fit(x_train, y_train)
    acc_scaler = knn_model.score(x_test, y_test)

    print(acc, acc_scaler)

    columns["model_name"].append(model_name)
    columns["rgb"].append(rgb)
    columns["size"].append(size)
    columns["normalization"].append(normalization)
    columns["use_train_val"].append(use_both)
    columns["accuracy"].append(acc)
    columns["accuracy_scaler"].append(acc_scaler)

  1%|          | 1/96 [00:06<10:54,  6.89s/it]

0.3238095238095238 0.3


  2%|▏         | 2/96 [00:10<07:47,  4.97s/it]

0.3 0.27380952380952384


  3%|▎         | 3/96 [00:14<07:02,  4.55s/it]

0.38095238095238093 0.36666666666666664
